# Baseline generation in TensorBoard

This script should generate baseline results in TensorBoard for easier reference.

In [2]:
import io
import os
import pandas as pd
from tensorboardX import SummaryWriter

In [3]:
RESULTS_DIR = "/home/ptempczyk/df/TorchSemiSeg/output"

## Data preparation

Data from [Table 1 and 2 in the paper](https://arxiv.org/pdf/2106.01226.pdf).

In [4]:
cps_results_csv = """
dataset,backbone,labeled_ratio,labeled_abs,nepochs_cps,MT,CCT,CutMix-Seg,GCT,CPS,CPS+CutMix
voc,res50v3+,16,662,32,66.77,65.22,68.90,64.05,68.21,71.98
voc,res50v3+,8,1323,34,70.78,70.87,70.70,70.47,73.20,73.67
voc,res50v3+,4,2646,40,73.22,73.43,72.46,73.45,74.24,74.90
voc,res50v3+,2,5291,60,75.41,74.75,74.49,75.20,75.91,76.15
voc,res101v3+,16,662,32,70.59,67.94,72.56,69.77,72.18,74.48
voc,res101v3+,8,1323,34,73.20,73.00,72.69,73.30,75.83,76.44
voc,res101v3+,4,2646,40,76.62,76.17,74.25,75.25,77.55,77.68
voc,res101v3+,2,5291,60,77.61,77.56,75.89,77.14,78.64,78.64
city,res50v3+,16,186,128,66.14,66.35,,65.81,69.79,74.47
city,res50v3+,8,372,137,72.03,72.46,,71.33,74.39,76.61
city,res50v3+,4,744,160,74.47,75.68,,75.30,76.85,77.83
city,res50v3+,2,1488,240,77.43,76.78,,77.09,78.64,78.77
city,res101v3+,16,186,128,68.08,69.64,,66.90,70.50,74.72
city,res101v3+,8,372,137,73.71,74.48,,72.96,75.71,77.62
city,res101v3+,4,744,160,76.53,76.35,,76.45,77.41,79.21
city,res101v3+,2,1488,240,78.59,78.29,,78.58,80.08,80.21
"""
df = pd.read_csv(io.StringIO(cps_results_csv))
df

,dataset,backbone,labeled_ratio,labeled_abs,nepochs_cps,MT,CCT,CutMix-Seg,GCT,CPS,CPS+CutMix
0,voc,res50v3+,16,662,32,66.77,65.22,68.90,64.05,68.21,71.98
1,voc,res50v3+,8,1323,34,70.78,70.87,70.70,70.47,73.20,73.67
2,voc,res50v3+,4,2646,40,73.22,73.43,72.46,73.45,74.24,74.90
3,voc,res50v3+,2,5291,60,75.41,74.75,74.49,75.20,75.91,76.15
4,voc,res101v3+,16,662,32,70.59,67.94,72.56,69.77,72.18,74.48
5,voc,res101v3+,8,1323,34,73.20,73.00,72.69,73.30,75.83,76.44
6,voc,res101v3+,4,2646,40,76.62,76.17,74.25,75.25,77.55,77.68
7,voc,res101v3+,2,5291,60,77.61,77.56,75.89,77.14,78.64,78.64
8,city,res50v3+,16,186,128,66.14,66.35,NaN,65.81,69.79,74.47
9,city,res50v3+,8,372,137,72.03,72.46,NaN,71.33,74.39,76.61


## Save results as TensorBoard logs

In [6]:
base_columns = ['dataset', 'backbone', 'labeled_ratio', 'labeled_abs', 'nepochs_cps']

for architecture in ["CPS", "CPS+CutMix"]:
    for index, row in df[base_columns + [architecture]].iterrows():
        path_string = f"{row['dataset']}8.{row['backbone']}.{architecture}/sup{row['labeled_ratio']}-baseline/tb/tb"
        with SummaryWriter(log_dir=os.path.join(RESULTS_DIR, path_string)) as tb:
            for step in range(int(row['nepochs_cps'])):
                tb.add_scalar('test/mIoU', float(row[architecture]), step)